In [1]:
import re
import sys; sys.path.append("../")

import numpy as np
import pandas as pd

from lib.columns import rearrange_complaint_columns
from lib.textract import parse_textract_datetime
from clean.baton_rouge_pd_cprr import clean_18

In [2]:
df = clean_18()
df

,investigation_status,action,disposition,department_code,department_desc,last_name,first_name,middle_initial,rule_code,paragraph_code,rule_violation,paragraph_violation,receive_year,receive_month,receive_day,occur_year,occur_month,occur_day,tracking_number
0,administrative review,exonerated,exonerated,P1254,cib street crimes unit,barcelona,jesse,r,3:17,40,carrying out orders,general orders (pursuit),2018,1,2,2017,12,30,2018-002
1,administrative review,not sustained,not sustained,P10280,cib narcotics,acree,jason,k,3:17,40,carrying out orders,general orders (pursuit),2018,1,9,2018,1,9,2018-004
2,office investigation,resigned,office investigation,P10476,op service comm center,rimes,melissa,j,2:4,66,abuse of sick leave,,2018,1,9,2017,12,3,2018-003
3,office investigation,office investigation,office investigation,P10476,op service comm center,rimes,melissa,j,2:4,66,abuse of sick leave,,2018,1,9,2017,12,3,2018-003
4,administrative review,not sustained,not sustained,P10578,cib cib,hamadeh,yuseff,g,3:20,53,use of force,hard empty hand,2018,1,10,2018,1,10,2018-005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,administrative review,letter of reprimand,sustained,P1245,patrol 2nd district,clark,kenneth,m,3:17,40,carrying out orders,general orders (pursuit),2019,1,30,2018,3,27,2019-003
145,administrative review,letter of reprimand,not sustained,P1245,patrol 2nd district,clark,kenneth,m,3:18,43,insubordination,,2019,1,30,2018,3,27,2019-003
146,administrative review,letter of reprimand,sustained,P1245,patrol 2nd district,clark,kenneth,m,2:12,21,conduct unbecoming an officer,,2019,1,30,2018,3,27,2019-003
147,administrative review,not sustained,not sustained,P10639,patrol 1st district,robertson,jason,r,3:17,40,carrying out orders,general orders (pursuit),2018,1,26,2018,1,26,2018-010


In [4]:
df.action.str.lower().unique()

array(['exonerated', 'not sustained', 'resigned', 'office investigation',
       'hold in abeyance', 'counseled', '', 'not sustained (60 day rule)',
       '30-day suspension', 'letter of caution',
       'command of temper (2-day suspension, verbal judo class, attaining respect class, early intervention',
       'tactical training in reasonable suspicion & felony stops',
       'tactical training in reasonable suspicion & felony slops',
       'letter of reprimand', 'peer intervention training',
       '21-day suspension', 'verbal judo training', '1-day suspension',
       'termination',
       'letter of caution; firearm safety training; range master',
       'exonerated: not sustained',
       'letter of caution & 1 day driving school',
       'letter of instruction', 'conference worksheet',
       '9-day suspension', '2-day suspension', '65-day suspension',
       'demotion - from sgt. to cpl.',
       '7 day suspension; de- escalation, r/s & p/c training; uof training',
       'aw

In [6]:
cprr["Disposition"].unique()

array(['Exonerated', 'Not Sustained', 'Office Investigation', 'Sustained',
       'Admin Review', ''], dtype=object)

In [2]:
import pandas as pd
pd.DataFrame.from_records(pd.Series([(3, 'a'), (2, 'b'), (1, 'c')]))

,0,1
0,3,a
1,2,b
2,1,c
